# Load modules

In [1]:
import os

from tqdm.auto import tqdm
from wgan_gp.models import Generator as WGAN_GENERATOR
from torch import nn
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
import numpy as np
import torch
import random
import torch.nn.functional as F
import torchvision
import pickle
import gc
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
from diffusers import DDPMPipeline

In [2]:
device = torch.device("cuda:0")

# GANs

## WGAN-GP

In [3]:
wgan_model = "WGAN-GP-v2_512"
wgan_path = os.path.join("GANs", wgan_model, "generator.pt")

In [4]:
generator = WGAN_GENERATOR(img_size=(64, 64, 3), latent_dim=512, dim=64).cuda()
generator.load_state_dict(torch.load(wgan_path))

<All keys matched successfully>

In [5]:
batch_size = 100
images_path = os.path.join("generated_images", wgan_model)
os.makedirs(images_path, exist_ok=True)
for i in range(10000 // batch_size):
    rand = generator.sample_latent(batch_size).to(device)
    output = generator(rand)
    for j in range(batch_size):
        image = transforms.ToPILImage()(output[j, ...])
        image.save(os.path.join(images_path, f"{i * batch_size + j}.png"))